# The Art and Science of Data Matching

## Mike Mull (@kwikstep)


If you have two sets of data items, A and B, each with no duplicates then 


$$
M=\{ (a,b)| a \in A, b \in B, a = b \}
$$

$$
U=\{ (a,b)| a \in A, b \in B, a \ne b \}
$$


By definition the union of the two sets M and U is the set of all pairs A and B, or:

$$
A \times B = M \cup U
$$


- Formally the "=" indicates a true match.  In most real data sources, some non-matches will be spuriously classified as in the M set and some true matches will be put in the U set.
- The vast majority of pairs in A x B will be in __U__.  If A and B both have 1000 members, A x B will have 1 million pairs, but __M__ can have at most 1000 elements.
- Ideally if (a,b) and (b,c) are in __M__ then (a,c) should be also.  However, there are some methods that do not result in this _transitive closure_ and so extra logic might be need to find transitive associations.
- In the deduplication case there is only one dataset (A) so you consider all of the pairs (a1,a2) in A x A except where a1 and a2 are the same item.  In this case the __M__ set contains the duplicates.  There may be numerous pairs that all refer to the same entity, so again extra work here is require to cluster the duplicates into groups and select the best attributes to retain.

<img src="Integrate.png">

<img src="Enrich.png">

## Data Exploration and Profiling

In [1]:
import pandas as pd
# Wage data, drop everything but unique name and addresses, and filter to only Florida.
df_oge = pd.read_csv('./data/ogesdw.whd.whisard.fl.csv')
df_oge_a_fl = df_oge[['trade_nm', 'legal_name', 'street_addr_1_txt', 'cty_nm', 'st_cd', 'zip_cd']].drop_duplicates()

print(type(df_oge_a_fl.trade_nm[0]))
df_oge_a_fl.info()

<class 'str'>
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16655 entries, 0 to 16702
Data columns (total 6 columns):
trade_nm             16655 non-null object
legal_name           16565 non-null object
street_addr_1_txt    16655 non-null object
cty_nm               16655 non-null object
st_cd                16655 non-null object
zip_cd               16655 non-null float64
dtypes: float64(1), object(5)
memory usage: 910.8+ KB


In [2]:
# Florida restaurant inspection data.  Drop everything but unique name and addresses
df_fl = pd.read_csv('./data/fl.restaurant-inspections.csv')
df_fl_a = df_fl[['dba', 'location_address', 'location_city', 'location_zip_code']].drop_duplicates()

df_fl_a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53576 entries, 0 to 130642
Data columns (total 4 columns):
dba                  53556 non-null object
location_address     53576 non-null object
location_city        53575 non-null object
location_zip_code    53568 non-null object
dtypes: object(4)
memory usage: 2.0+ MB


In [3]:
t={ord(x): y for (x, y) in zip('0123456789', '9999999999')}

df_fl_a['pzip'] =  df_fl_a.location_zip_code.apply(lambda x: x.translate(t) if pd.notnull(x) else x)
print(df_fl_a.groupby('pzip').size())

df_oge_a_fl['pzip'] =  df_oge_a_fl.zip_cd.apply(lambda x: str(int(x)).translate(t) if pd.notnull(x) else x)
print(df_oge_a_fl.groupby('pzip').size())

pzip
99999         47778
99999-9999     1611
999999999      4179
dtype: int64
pzip
99999    16655
dtype: int64


In [4]:
import re
def leading_patt(s):
    p = (('[A-Z][a-z]', 'Aa'), ('[A-Z][A-Z]', 'AA'), ('[0-9][0-9]', '99'))
    for rx, patt in p:
        if re.search(rx, s):
            return patt
    return s

df_oge_a_fl['patt'] = df_oge_a_fl.trade_nm.apply(leading_patt)
df_oge_a_fl.groupby('patt').size()

patt
4C                          1
7-eleven                    1
A.R.M.                      1
AA                        350
Aa                      16295
G4S                         1
H. A. R. T.                 1
I.P.S.                      1
I.T.M.                      1
S & L                       1
krystel's restaurant        1
turner construction         1
dtype: int64

In [5]:
df_fl_a['patt'] = df_fl_a.dba.apply(lambda x: leading_patt(x) if pd.notnull(x) else 'NONE')
df_fl_a.groupby('patt').size()

patt
4D               1
99              39
A I O L I        1
A&W              1
AA           53505
D J'S            1
I H O P          1
J&L              1
NONE            20
P R'S            1
R J'S            1
S 3              1
T S I            1
U K              1
Z'S R & B        1
dtype: int64

In [6]:
# Process name and zip code fields
df_fl_a['nname'] = df_fl_a.dba.apply(lambda x:x.lower().translate({None:"'.,"}) if pd.notnull(x) else '')
df_oge_a_fl['nname'] = df_oge_a_fl.trade_nm.apply(lambda x:x.lower().translate({None:"'.,"}) if pd.notnull(x) else '')
df_fl_a['nzip'] = df_fl_a.location_zip_code.apply(lambda x:x[0:5] if pd.notnull(x) else x)
df_oge_a_fl['nzip'] = df_oge_a_fl.zip_cd.apply(lambda x: str(int(x)))

# Merge on name and zip
dfmz = df_fl_a.merge(df_oge_a_fl, on=['nname', 'nzip'])
dfmz[['nname','nzip', 'location_address', 'street_addr_1_txt']].sort('nname').head(10)

,nname,nzip,location_address,street_addr_1_txt
551,15 south ristorante enoteca,34236,15 S BLVD OF PRESIDENTS,15 South Blvd. of Presidents
334,5th avenue bistro,32809,8001 S ORANGE BLOSSOM TRAIL,8001 South Orange Blossom Trail STE 968
91,84 diner,33325,11432 W SR 84,11432 St Rd 84
169,acropolis greek taverna,33605,1833 E 7 AVE,1833 E. 7th Avenue
418,al's pizza,32223,11190 SAN JOSE BLVD,11190 San Jose Blvd
353,alaqua country club,32779,2091 ALAQUA DR,3060 Players Point
391,american grill,32202,2 INDEPENDENT DR #201,2 Independent Drive
434,amici italian restaurant,32080,1915 S AIA B,1915 B A1A South
70,amici's trattoria italiana,33196,10201 HAMMOCKS BLVD SUITE 137,10201 Hammocks Boulevard
82,anglins beach cafe,33308,2 COMMERCIAL BLVD,2 Commercial Blvd.


In [7]:
dfmz.groupby(['nname','nzip']).filter(lambda x: len(x['cty_nm']) > 1)[['nname',
                                                                      'nzip', 
                                                                      'location_address', 
                                                                      'street_addr_1_txt']].head(10)

,nname,nzip,location_address,street_addr_1_txt
17,il bolognese,33139,626 - 630 LINCOLN RD,1052 Ocean Drive
18,il bolognese,33139,626 - 630 LINCOLN RD,626 Lincoln Rd.
19,il bolognese,33139,1400 OCEAN DRIVE,1052 Ocean Drive
20,il bolognese,33139,1400 OCEAN DRIVE,626 Lincoln Rd.
23,wajiro's restaurant,33184,12670 SW 8 ST,12670 SW 8th Street
24,wajiro's restaurant,33184,12670 SW 8 ST,12670 S.W. 8 Street
29,hosteria romana,33139,429 ESPANOLA WAY,429 Espaola Way
30,hosteria romana,33139,429 ESPANOLA WAY,435 Espanola Way #B
38,johnny rockets,33172,11401 NW 12 ST #E-508,11401 N W 12th Street
39,johnny rockets,33172,1455 NW 107 AVE #454 A,11401 N W 12th Street


In [8]:
df_fl_a.groupby(['nname','nzip']).filter(lambda x: len(x['dba']) > 2).sort('dba').head(10)

,dba,location_address,location_city,location_zip_code,pzip,patt,nname,nzip
87993,ATLANTIC VENDORS INC,3111 S ATLANTIC AVE,DAYTONA BEACH SHORES,32118,99999,AA,atlantic vendors inc,32118
87994,ATLANTIC VENDORS INC,2225 S ATLANTIC AVE,DAYTONA BCH SHORES,32118,99999,AA,atlantic vendors inc,32118
87996,ATLANTIC VENDORS INC,2411 S ATLANTIC AVE,DAYTONA BCH SHORES,32118,99999,AA,atlantic vendors inc,32118
87997,ATLANTIC VENDORS INC,3125 S ATLANTIC AVE,DAYTONA BCH SHORES,32118,99999,AA,atlantic vendors inc,32118
87999,ATLANTIC VENDORS INC,400 N ATLANTIC AVE,DAYTONA BCH,32118,99999,AA,atlantic vendors inc,32118
88045,ATLANTIC VENDORS INC,2209 S ATLANTIC AVE,DAYTONA BCH SHORES,32118,99999,AA,atlantic vendors inc,32118
88061,ATLANTIC VENDORS INC,2125 S ATLANTIC AVE,DAYTONA BCH,32118,99999,AA,atlantic vendors inc,32118
88291,ATLANTIC VENDORS INC,2209 S ATLANTIC AVE,DAYTONA BEACH SHORES,32118,99999,AA,atlantic vendors inc,32118
88474,ATLANTIC VENDORS INC,1615 S ATLANTIC AVE,DAYTONA BEACH,32118,99999,AA,atlantic vendors inc,32118
88493,ATLANTIC VENDORS INC,2500 N ATLANTIC AVE,DAYTONA BEACH,32118,99999,AA,atlantic vendors inc,32118


In [9]:
# Merge on name and zip
dfz = df_fl_a.merge(df_oge_a_fl, on='nzip')
dfz = dfz[dfz.nname_x != dfz.nname_y]
dfz.head()

,dba,location_address,location_city,location_zip_code,pzip_x,patt_x,nname_x,nzip,trade_nm,legal_name,street_addr_1_txt,cty_nm,st_cd,zip_cd,pzip_y,patt_y,nname_y
0,SPS RESTAURANT,1757 NE 2 AVE,MIAMI,331321191,999999999,AA,sps restaurant,33132,KITCHEN TM SYSTEM,TM SYSTEMS LLC,"1550 Biscayne Blvd, 3rd Floor",Miami,FL,33132,99999,AA,kitchen tm system
1,SPS RESTAURANT,1757 NE 2 AVE,MIAMI,331321191,999999999,AA,sps restaurant,33132,The Knife Restaurant (Bayside),Carma Cinco LLC,401 Biscayne Blvd,Miami,FL,33132,99999,Aa,the knife restaurant (bayside)
2,SPS RESTAURANT,1757 NE 2 AVE,MIAMI,331321191,999999999,AA,sps restaurant,33132,Latin American Cafeteria at Bayside,"LATIN AMERICAN BAYSIDE, INC.",401 Biscayne Blvd.,Miami,FL,33132,99999,Aa,latin american cafeteria at bayside
3,SPS RESTAURANT,1757 NE 2 AVE,MIAMI,331321191,999999999,AA,sps restaurant,33132,Mar y Tierra,Adrian Mar & Tierra Corp,266 NE 1st Street,Miami,FL,33132,99999,Aa,mar y tierra
4,SPS RESTAURANT,1757 NE 2 AVE,MIAMI,331321191,999999999,AA,sps restaurant,33132,Miss Yip,Chinese Cafe 900 LLC,900 Biscayne Blvd,Miami,FL,33132,99999,Aa,miss yip


In [10]:
import jellyfish

dfz['ed'] = dfz[['nname_x', 'nname_y']].apply(lambda x: jellyfish.levenshtein_distance(x[0],x[1]), axis=1)
dfz.groupby('ed').size().head()

ed
1    139
2     70
3     87
4    276
5    651
dtype: int64

In [11]:
dfz[dfz.ed<3][['nname_x', 'nname_y']].head(10)

,nname_x,nname_y
3059,prelude by barton g,prelude by garton g
36412,mcdonalds #6589,mcdonald's #6589
52831,new cafe,sea cafe
53484,the palace,the place
56756,cafe milano,caffe milano
63884,yuca restaurant,yuca's restaurant
65200,vino's pizza & brew,gino's pizza & brew
68380,barton g the restaurant,barton g- the restaurant
75351,aura restaurant,guru restaurant
75703,finnegans way,finnegan's way


## Comparing Fields

$$
Sim(r_{A}, r_{B})
$$

In practice we compare against attributes of rA or rB

$$
Sim(a_{1}, a_{2}..., a_{n}; b_{1}, b_{2}, ..., b_{n}) = [s_{1}(a_{1}, b_{1}), ..., s_{n}(a_{n}, b_{n})]
$$

So we end up with a comparison vector for each record.

### Preprocessing and Transformation

- Simple text transformations: lowerize, remove punctuation
- More complex:
  - Replace patterns ('&' -> 'and')
  - In some situations, remove stop words, use stemming
- Standard zip length or format
- Imputation


### Approximate String Comparison

- NLTK
  - Edit distance
  - Jaccard distance
- Difflib
  - SequenceMatcher
- Jellyfish
  - __Levenshtein__ 
  - Jaro and __Jaro-Winkler__
  - Soundex
- FuzzyWuzzy

- Token-Based
  - __Jaccard similarity__
  - __SoftTf-Idf__


### Levenshtein/Edit Distance

In [12]:
from jellyfish import levenshtein_distance
print(levenshtein_distance('pydata', 'padata'))    # 1-character change
print(levenshtein_distance('pydata', 'pandata'))   # Insert character
print(levenshtein_distance('pydata', 'pandasa'))   # 1-character change
print(levenshtein_distance('pydata', 'pandas'))    # Character deletion

1
2
3
4


### Jaro/Jaro-Winkler

$$
Jaro(s_{1}, s_{2}) = w_{1}\frac{c}{L_{1}} + w_{2}\frac{c}{L_{2}} + w_{t}\frac{c-\tau}{c}
$$
where:
$$
L_{1} \mbox{ is length of }s_{1} \\
L_{2} \mbox{ is length of }s_{2} \\
c \mbox{ is number of characters in common } \\
\tau \mbox{ is number of transpositions } \\
$$

$$
JaroWinkler(s_{1}, s_{2}) = Jaro(s_{1}, s_{2}) + 0.1 i(1 - Jaro(s_{1}, s_{2}))
$$
where:
__i__ is min(4, number of initial characters that match)

### Jaccard

In [26]:
def jaccard(s1, s2):
    ''' Jaccard similarity score.  (1 - jaccard) is a valid distance metric '''
    a = set(s1)
    b = set(s2)
    if len(a) == 0 and len(b) == 0:
        return 0
    return len(a & b) / len(a | b)

print(jaccard('pydata', 'pandas'))
print(jaccard(['bag', 'of', 'words'], ['words', 'bag']))

0.42857142857142855
0.6666666666666666


In [27]:
# N-gram function courtesy of Peter Norvig
def ngrams(seq, n):
    "List all the (overlapping) ngrams in a sequence."
    return [seq[i:i+n] for i in range(1+len(seq)-n)]

print(jaccard(ngrams('pydata', 2), ngrams('pandas', 2)))
print(jaccard(ngrams('pydata', 3), ngrams('pandas', 3)))

0.1111111111111111
0.0


### SoftTf-Idf

- _SoftTf-Idf_ is a variation of Tf-Idf (_Term Frequency-Inverse Document Frequency_) that includes similar tokens


$$
idf(t,D) = log \frac{|D|}{| d \in D, t \in d |}
$$

-  In the normal Td-Idf scheme only tokens that are the same in two documents matter.

$$
\{ sim(w,v) > \theta | w \in X, v \in Y \} 
$$


## Human Names

|  Name |  Source  |
| --- |  --- |
| RICHARD ALLAN BREGGER | Finra |
| RICHARD A BREGGER | DOL |
| BREGGER RICHARD A | DOL |
| RICHAPT) A BREGGER | DOL (OCR error) |
| RICHARD A BREGGER AXA FINANCIAL | DOL |
| RICHARD D BREGGER | DOL |
| RICHARDBREGGER | Record Keeper |

<img src='bregger.png'>

### Stupid Human (Name) Tricks

- Almost always want to parse names into (first, middle or MI, last)
- Do not use regex
- Substituting first names is helpful, but build your own database of alternatives
- Do not use regex
- As you match, retain alternatives and do look-ups

### The _nameparser_ library

In [24]:
from nameparser import HumanName

HumanName('DR. RICHARD ALLAN "Rick" BREGGER, Ph.D')

<HumanName : [
	title: 'DR.' 
	first: 'RICHARD' 
	middle: 'ALLAN' 
	last: 'BREGGER' 
	suffix: 'Ph.D'
	nickname: 'Rick'
]>

In [16]:
HumanName('BREGGER, RICHARD A')

<HumanName : [
	title: '' 
	first: 'RICHARD' 
	middle: 'A' 
	last: 'BREGGER' 
	suffix: ''
	nickname: ''
]>

In [25]:
HumanName('BREGGER RICHARD A')

<HumanName : [
	title: '' 
	first: 'BREGGER' 
	middle: 'RICHARD' 
	last: 'A' 
	suffix: ''
	nickname: ''
]>

## Addresses and Geocoding

- Addresses are more frequently separated into multiple fields, usually street address, city, state, zip code in the source data.
- Zip codes have 3,5, and 9 (Zip+4) variants
- Since Zip codes are designed for efficiency of mail delivery, they aren't a consistent indicator of physical proximity.
- Different businesses might have identical addresses (aka, "the food-court syndrome")

In [17]:
df_fl_a[df_fl_a.location_address=='39 S BENEVA RD']

,dba,location_address,location_city,location_zip_code,pzip,patt,nname,nzip
129483,HOSTED GOURMET,39 S BENEVA RD,SARASOTA,34232,99999,AA,hosted gourmet,34232
129503,STAMPER CHEESE COMPANY,39 S BENEVA RD,SARASOTA,34232,99999,AA,stamper cheese company,34232
129507,MOUTHOLE BBQ,39 S BENEVA RD,SARASOTA,34232,99999,AA,mouthole bbq,34232
129515,PALEO CONNECTION LLC,39 S BENEVA RD,SARASOTA,34232,99999,AA,paleo connection llc,34232
129516,DOLCE VITA LLC,39 S BENEVA RD,SARASOTA,34232,99999,AA,dolce vita llc,34232
129517,BRICKS BBQ,39 S BENEVA RD,SARASOTA,34232,99999,AA,bricks bbq,34232
129521,WINGS ON WHEELS LLC,39 S BENEVA RD,SARASOTA,34232,99999,AA,wings on wheels llc,34232
130612,MOBILE BISTRO,39 S BENEVA RD,SARASOTA,34240,99999,AA,mobile bistro,34240
130625,RI QUISIMO,39 S BENEVA RD,SARASOTA,34232,99999,AA,ri quisimo,34232


## Blocking

In [18]:
df_fl_a['k'] = 1
df_oge_a_fl['k'] = 1
df_block = df_fl_a.sort('nname').iloc[0:100].merge(df_oge_a_fl.sort('nname').iloc[0:100], 
                                                  on='k')#[['index_x', 'nname_x', 'index_y', 'nname_y']]


## Classifying Matches

### Simple Scoring

Recall that for each record pair we're supposing that we have a vector where each element of the vector is a comparison of the comparable attributes:
$$
[s_{1}(a_{1}, b_{1}), ..., s_{n}(a_{n}, b_{n})]
$$

- Sum the individual scores and set a threshold T1.  Anything > T1 is a match
- For bonus points, choose a second threshold T2 so that anything < T2 is a non-match
- Any sum between T1 and T2 needs review.


### Rules

### Probabilistic Matching (Fellugi-Sunter)

Suppose our comparison vector is:

$$
  \gamma(r_{A}, r_{B}) = [JaroWinkler(Name_{A}, Name_{B}) > 0.9, (Zip_{A} = Zip_{B}), Levenshtein(StreetNum_{A}, StreetNum_{B}) < 2]
$$

We'll call this a _comparison vector_ or an _agreement pattern_.  The are 8 separate patterns (2^3) in this case


In [19]:
import itertools
pd.DataFrame(data=list(itertools.product(['match','unmatch'], repeat=3)),
             columns=['name', 'zip', 'street num'])

,name,zip,street num
0,match,match,match
1,match,match,unmatch
2,match,unmatch,match
3,match,unmatch,unmatch
4,unmatch,match,match
5,unmatch,match,unmatch
6,unmatch,unmatch,match
7,unmatch,unmatch,unmatch


The method then defines the following two conditonal probabilities:

$$
\begin{align}
m(\gamma) = P[\gamma(r_{A}, r_{B})| (r_{A}, r_{B}) \in M] \\
= \sum_{a,b \in M} P[\gamma(r_{A},r_{B})]P[(r_{A}, r_{B}) \in M]
\end{align}
$$
and
$$
\begin{align}
u(\gamma) = P[\gamma(r_{A}, r_{B})| (r_{A}, r_{B}) \in U] \\
= \sum_{a,b \in U} P[\gamma(r_{A},r_{B}]P[(r_{A}, r_{B}) \in U]
\end{align}
$$


The Fellegi-Sunter method results in a _linkage rule_:

$$
	\begin{array}{ll}
		\frac{m}{u} > T_{\mu}  & \mbox{match}  \\
        T_{\mu} < \frac{m}{u} < T_{\lambda}  & \mbox{potential match}  \\
		\frac{m}{u} < T_{\lambda}  & \mbox{non-match}  \\
	\end{array}
$$


But Wait!

- By definition we don't know which records are in M and which are in U
- In some cases __m/u__ can be estimated, and then progressively refined.
- Most general method is Expectation-Maximization algorithm

- Assume distribution of patterns is a mixture of distributions, with parameters:

$$
\Phi = ((m_{1}\dots m_{k}), (u_{1}\dots u_{k}), p)
$$

- Also assume there is a _latent variable_ __g__ that determines whether a record pair is in __M__ or __U__.  Then the likelihood function for the data we have is:


$$
f(g,\gamma; \Phi) = \prod_{j=1}({pP(\gamma_{j}|r_{j} \in M))^g((1-p)P(\gamma_{j}|r_{j} \in U)})^{(1-g)}
$$


### Supervised Learning

- Requires training data where matches/non-matches are already known
- Always imbalanced since there are many more non-matches than matches

In [20]:
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier

df_matches = pd.read_csv('./data/matches.csv')

df_matches['snum_dist'] = df_matches[['snum_x', 'snum_y']].apply(lambda s: levenshtein_distance(str(s[0]), str(s[1])),
                                                                 axis=1)
df_matches['city_sim'] = df_matches[['cty_nm', 'location_city']].apply(lambda c: jaccard( ngrams(c[0].lower(),2),
                                                                                          ngrams(c[1].lower(),2)),
                                                                      axis=1)
df_matched = df_matches[df_matches.match==1]
df_unmatched = df_matches[df_matches.match==0]
df_bal = pd.concat( [df_matched, 
                     df_unmatched.sample(len(df_matched)) ] ).reset_index()

y = df_bal.match
X = df_bal[['jw', 'snum_dist', 'city_sim']].as_matrix()

kf = KFold(len(y),n_folds=5,shuffle=True)
y_pred = y.copy()

clf = RandomForestClassifier()

# Iterate through folds
for train_index, test_index in kf:
    X_train, X_test = X[train_index], X[test_index]
    y_train = y[train_index]
    # Initialize a classifier with key word arguments

    clf.fit(X_train,y_train)
    y_pred[test_index] = clf.predict(X_test)

print(np.mean(y == y_pred))

0.998106060606
